In [6]:
import gust
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy
import scipy.sparse as sp
import torch.distributions as dist
from time import time

from scipy.spatial.distance import squareform

In [7]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

A, X, _, z = gust.load_dataset('cora_ml').standardize().unpack()
#adj = torch.FloatTensor(A.toarray()).cuda()
#A = A[0:4,0:4]

In [36]:
def laplacian(A):
    #Transition Matrix P=D-A
    num_nodes = A.shape[0]
    D = np.ravel(A.sum(1))
    L = sp.diags(D) - A
    return L


def sym_normalized_laplacian(A):
    #Symmetric, Normalized Laplacian P=D^(−1/2)AD^(−1/2)
    num_nodes = A.shape[0]
    D = np.ravel(A.sum(1))
    #D[D == 0] = 1  # avoid division by 0 error
    D_sqrt = np.sqrt(D)
    a=np.ones(D_sqrt.shape[0])
    D_sqrt_inv = np.divide(a, D_sqrt, out=np.zeros_like(a), where=D!=0) 
    L = sp.diags(D_sqrt_inv) * A * sp.diags(D_sqrt_inv)
    #L = A / D_sqrt[:, None] / D_sqrt[None, :]
    return L

def Transition(A):
    #Laplacian P=D^−1A
    num_nodes = A.shape[0]
    D = np.ravel(A.sum(1))
    #D[D == 0] = 1  # avoid division by 0 error
    a=np.ones(D.shape[0])
    D_inv = np.divide(a, D, out=np.zeros_like(a), where=D!=0)
    L = sp.diags(D_inv) * A
    return L

def PPR(A):
    #Personalized PageRank Matrix as described in https://openreview.net/pdf?id=H1gL-2A9Ym with the there used hyperparameter alpha=0.1
    #P=alpha(I-(1-alpha)*D^-1/2(A+I)D^-1/2)^-1
    print(A.toarray())
    alpha = 0.1  
    num_nodes = A.shape[0]
    D = np.ravel(A.sum(1))
    #D[D == 0] = 1  # avoid division by 0 error
    D_sqrt = np.sqrt(D)
    a=np.ones(D_sqrt.shape[0])
    D_sqrt_inv = np.divide(a, D_sqrt, out=np.zeros_like(a), where=D!=0)
    A_tilde = sp.diags(D_sqrt_inv) * (A + sp.identity(A.shape[0])) * sp.diags(D_sqrt_inv)
    print('A_tilde: ', A_tilde.toarray())
    L_inv = (sp.identity(A.shape[0]) - (1-alpha) * A_tilde)
    print('L_inv: ', L_inv.toarray())
    L = alpha * np.linalg.pinv(L_inv.toarray())
    print(L)
    return L

def NetMF(A):
    eps=1e-5
    #volume of the graph, usually for weighted graphs, here weight 1
    vol = A.sum()
    
    #b is the number of negative samples, hyperparameter
    b = 3
    
    #T is the window size, as a small window size algorithm is used, set T=10, which showed the best results in the paper
    T=10
    
    #Transition Matrix P=D^-1A
    num_nodes = A.shape[0]
    D = np.ravel(A.sum(1))
    #D[D == 0] = 1  # avoid division by 0 error
    a=np.ones(D.shape[0])
    D_inv = np.divide(a, D, out=np.zeros_like(a), where=D!=0)
    P = np.diag(D_inv) * A.todense()
    
    #Compute M = vol(G)/bT (sum_r=1^T P^r)D^-1
    sum_np=0
    for r in range(1,T+1):
        sum_np+=np.linalg.matrix_power(P,r)
    M = sum_np * np.diag(D_inv) * vol / (b*T)
    M_max = np.maximum(M,np.ones(M.shape[0]))

    #Compute SVD of M
    u, s, vh = np.linalg.svd(np.log(M_max), full_matrices=True)

    #Compute L
    L = u*np.diag(np.sqrt(s+eps))
    print(L.sum(axis=1))
    return L


NetMF(A)

sum:  [[3.20058523e-01 1.70155936e-05 7.70422454e-05 ... 5.96711938e-04
  4.39552749e-05 3.28859651e-05]
 [7.29239725e-06 4.60282688e-01 2.99302856e-01 ... 1.19440768e-04
  2.06799247e-04 3.63538114e-05]
 [3.30181052e-05 2.99302856e-01 2.71628466e-01 ... 2.05855326e-04
  2.64352828e-04 9.01183721e-05]
 ...
 [8.95067908e-04 4.18042687e-04 7.20493640e-04 ... 9.25508086e-02
  1.14756111e-04 6.17825482e-04]
 [4.39552749e-05 4.82531576e-04 6.16823265e-04 ... 7.65040742e-05
  9.98571588e-02 3.82944497e-04]
 [9.86578954e-05 2.54476679e-04 6.30828605e-04 ... 1.23565096e-03
  1.14883349e-03 7.66977048e-02]]
M_max:  [[56.76415708  1.          1.         ...  1.          1.
   1.        ]
 [ 1.         34.98586791 22.74986756 ...  1.          1.
   1.        ]
 [ 1.         22.74986756 20.64635033 ...  1.          1.
   1.        ]
 ...
 [ 1.          1.          1.         ... 24.6216001   1.
   1.        ]
 [ 1.          1.          1.         ...  1.         17.71022187
   1.        ]
 [ 1.   

matrix([[ 1.11958942e-02, -9.95638662e-04,  1.01688372e-02, ...,
         -1.61217945e-19, -5.16886670e-20,  4.00235886e-20],
        [ 1.93902934e-02,  3.02142575e-03,  1.07350619e-02, ...,
         -7.78848217e-19, -3.01348150e-19,  1.52764129e-19],
        [ 1.97666083e-02,  3.89060202e-03,  9.37555307e-03, ...,
          1.06068750e-18,  4.08071585e-19, -2.20336528e-19],
        ...,
        [ 1.23470922e-01, -1.49687788e-02, -4.89413386e-03, ...,
          9.45411483e-19,  3.48215956e-19, -1.83262604e-19],
        [ 1.75006227e-02, -4.35977926e-03,  3.68883715e-02, ...,
          3.59512755e-19,  1.20170618e-19, -8.47842498e-20],
        [ 2.27091134e-02,  1.89522596e-03,  3.34534428e-04, ...,
          6.19152968e-19,  2.35249307e-19, -1.21058559e-19]])

In [3]:
N = A.shape[0]
D = 32

Z = nn.Parameter(torch.empty(N, D).normal_(std=0.1))
x = nn.Parameter(torch.empty(N, D).normal_(std=0.1))

opt = torch.optim.Adam([Z], lr=1e-2)
e1, e2 = A.nonzero()

In [4]:
def L0(A):    
    return torch.FloatTensor(A.toarray()).cuda()

In [5]:
def L1(A):
    #L=D^-1A
    degreenp=A.sum(axis=1)
    degree = torch.from_numpy(degreenp).cuda().view(-1)
    degreematrix = torch.zeros(N,N)
    degreematrix[np.diag_indices(N)]=degree
    invdegree = torch.inverse(degreematrix)
    adj = torch.FloatTensor(A.toarray()).cuda()
    return torch.matmul(invdegree, adj)

Z = L1(A)

In [23]:
def L2(A):
    degreenp=A.sum(axis=1)
    degree = torch.from_numpy(degreenp).cuda().view(-1)
    degreematrix = torch.zeros(N,N)
    degreematrix[np.diag_indices(N)]=degree
    sqrtdegree = degreematrix.sqrt()
    invdegree = torch.inverse(sqrtdegree)
    adj = torch.FloatTensor(A.toarray()).cuda() 
    return torch.matmul(invdegree, torch.matmul(adj, invdegree))

In [24]:
L2(A)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1429,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1429, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [20]:
def sig(Z, b=0.1, eps=1e-8): 
    dist = torch.matmul(Z,Z.T) +b
    sigdist = 1/(1+torch.exp(dist+eps)+eps)
    logsigdist = torch.log(sigdist+eps)
    pos_term = logsigdist[e1,e2]
    neg_term = torch.log(1-sigdist)
    neg_term[np.diag_indices(N)] = 0.0
    
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [21]:
def dist(Z, eps=1e-5):
    gamma = 0.1
    dist = ((Z[:, None] - Z[None, :]).pow(2.0).sum(-1) + eps).sqrt()
    neg_term = torch.log(-torch.expm1(-dist)*gamma + eps)
    print(neg_term)
    neg_term[np.diag_indices(N)] = 0.0
    pos_term = -dist[e1, e2]   
    neg_term[e1, e2] = 0.0
    
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [22]:
def exp(Z, eps=1e-8):
    #e1, e2 = similarity_measure.nonzero()
    emb_abs = torch.FloatTensor.abs(Z)
    dist = -torch.matmul(emb_abs, emb_abs.T)
    neg_term = dist
    neg_term[np.diag_indices(Z.shape[0])] = 0.0
    expdist = torch.exp(dist)
    embedding = 1 - expdist
    logdist = torch.log(embedding + eps)
    pos_term = logdist[e1, e2]
    size=Z.shape[0]
    print(embedding)
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [23]:
def kl(L, Z, eps=1e-8):
    #P=softmax(ZZ^T)
    dist=torch.matmul(Z,Z.T)
    sigdist = 1/(1+torch.exp(dist+eps)+eps)
    logsigdist = torch.log(sigdist+eps)
    losses = T*logsigdist
    return losses.sum()

In [24]:
for epoch in range(3):
    opt.zero_grad()
    loss = exp(Z)
    loss.backward()
    opt.step()
    print(loss.item())

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0202,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0202, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn